# Simulations to demonstrate Filament Homing Function

In [102]:
import sys
if 'init_modules' in globals().keys():
    # second or subsequent run: remove all but initially loaded modules
    for m in sys.modules.keys():
        if m not in init_modules:
            del(sys.modules[m])
else:
    # first run: find out which modules were initially loaded
    init_modules = sys.modules.keys()
    
import numpy as np
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import interpolate
import h5py


import os
from sys import platform

from joblib import Parallel, delayed
import multiprocessing

import pyfilaments.analysisutils as analysis
from pyfilaments.activeFilaments import activeFilament

import imp
imp.reload(analysis)
%matplotlib notebook
plt.close('all')

# Figure parameters
from matplotlib import rcParams
from matplotlib import rc
from matplotlib import cm
from tqdm import tqdm

In [103]:
# Choose a simulation to use to generate initial filament shapes
file = '/home/deepak/ActiveFilamentsSearch_backup_3/ProgrammingFilamentBehavior/Homing/SimResults_Np_32_Shape_line at angle_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.718_activityTime_750_simType_point/SimResults_00_.hdf5'
fil = analysis.analysisTools(file = file)
homing_folder = os.path.join(fil.rootFolder, 'HomingBehavior_activityStrength')
    
fil.homing_folder = homing_folder

Attribute not found


In [104]:
import random 
# Global parameters
N_transient = 400 # No:of activity cycles for transients to decay. 

N_IC_time = 10 # IC based on different activity cycles/time
N_IC_phase = 10 # Different IC within each activity phase


def choose_ic_time_phase(fil):
    """Takes a filament simulation and returns randomly chosen time points at a randomly chosen phase
        along the activity cycle.
    """
    
    # Total activity cycles in the simulation
    N_cycles = int(fil.Time[-1]/fil.activity_timescale)

    # Choose n random activity cycles (time)
    n_cycles = random.sample(range(N_transient, N_cycles), N_IC_time)

    print(n_cycles)

    # Within each activity cycle choose points evenly distributed in phase
    n_phase = random.sample(list(np.linspace(0, 2*np.pi, 100)), N_IC_phase)

    print(n_phase)

    # Compute the minimum resolved phase difference in the simulations
    delta_phase = 2*np.pi*np.mean(fil.Time[1:] - fil.Time[:-1])/fil.activity_timescale # Smallest change in phase

    time_points = np.array(range(0, len(fil.Time)))
    # Find the index corresponding to each unique IC

    # Create empty array to store indices for each unique IC

    ic_mesh = np.empty((N_IC_time, N_IC_phase), dtype = int)

    # Phase array for the whole simulation
    phase_array = 2*np.pi*(fil.Time%fil.activity_timescale)/fil.activity_timescale


    for ii, phase in enumerate(n_phase):

        print(phase)
        print(fil.derived_data['Phase'])
        abs_val_array = np.abs(fil.derived_data['Phase'] - phase)
        constant_phase_mask = abs_val_array <= 0.5*delta_phase
        time_indices = time_points[constant_phase_mask]
        print(time_indices)
        ic_mesh[:, ii] = time_indices[n_cycles]
        
    return ic_mesh

In [105]:
def generate_save_ic(fil = None, ic_mesh = None):
    """ Start with a filament shape at the randomly chosen time-points and store these shapes as ICs 
        as .hdf5 files in the IC_analysis folder within the simulation data folder.
        Returns: Nothing
    """
    if not(os.path.exists(fil.homing_folder)):
           os.makedirs(fil.homing_folder)

    filament_init_shape = np.zeros(fil.Np*fil.dim, dtype = np.double)

    
    phase_array = 2*np.pi*(fil.Time%fil.activity_timescale)/fil.activity_timescale

    
    for ii in range(N_IC_time):

        for jj in range(N_IC_phase):

            time_index = ic_mesh[ii, jj]

            phase = phase_array[time_index]

            ic_folder = os.path.join(fil.homing_folder, 'IC_{}'.format(time_index))

            if(not os.path.exists(ic_folder)):
                os.makedirs(ic_folder)

            filament_shape = fil.R[time_index,:]
            
            
            
#             plt.figure(figsize=(2,2))
#             fil.plotFilament(r = filament_shape)
#             plt.title(time_index)
#             plt.axis('equal')

    
            save_file = 'filament_ic_{0:05d}.hdf5'.format(time_index)
            # Save the resulting filament shapes to disk
            with h5py.File(os.path.join(ic_folder, save_file), "w") as f:

                dset = f.create_group("simulation data")
                dset.create_dataset("Initial Position", data = filament_shape)
                dset.attrs['Time index'] = time_index
                # Also save the phase value of the simulation
                dset.attrs['Phase'] = phase

In [106]:
ic_mesh = choose_ic_time_phase(fil)
generate_save_ic(fil = fil, ic_mesh= ic_mesh)

[462, 457, 461, 420, 466, 470, 418, 481, 455, 469]
[4.569589314312426, 4.886921905584122, 5.267721015110158, 0.25386607301735703, 3.6175915404973376, 5.648520124636194, 1.0154642920694281, 1.5231964381041423, 1.71359599286716, 1.269330365086785]
4.569589314312426
[0.         0.0837758  0.16755161 ... 6.1156337  6.1994095  0.        ]
[   55   130   205   280   355   430   505   580   655   730   805   880
   955  1030  1105  1180  1255  1330  1405  1480  1555  1630  1705  1780
  1855  1930  2005  2080  2155  2230  2305  2380  2455  2530  2605  2680
  2755  2830  2905  2980  3055  3130  3205  3280  3355  3430  3505  3580
  3655  3730  3805  3880  3955  4030  4105  4180  4255  4330  4405  4480
  4555  4630  4705  4780  4855  4930  5005  5080  5155  5230  5305  5380
  5455  5530  5605  5680  5755  5830  5905  5980  6055  6130  6205  6280
  6355  6430  6505  6580  6655  6730  6805  6880  6955  7030  7105  7180
  7255  7330  7405  7480  7555  7630  7705  7780  7855  7930  8005  8080
  8155 

In [107]:
def create_batchsim_table(folders = None, fil = None):
    
    """ Create a table/dict of IC for filament shape along with phase info that can be passed to run a Parallel job
        
        Input: Folder containing IC files. This folder should contain IC that are closely spaced together. 
        
        Returns: ic_list (dict containing: path, filename, filament shape and phase of each IC)
    
    """
    # Create a dictionary to store all IC we want to simulate
    ic_table = {}
    ic_counter = 0
    
    root_folder = fil.homing_folder


    for folder in folders:

        ic_files = os.listdir(os.path.join(root_folder, folder))
     
        for file in ic_files:
            
            if(file.endswith('.hdf5') and 'ic' in file):
                with h5py.File(os.path.join(root_folder, folder, file), "r") as f:
                    if('simulation data' in f.keys()): # Load the simulation data
                        dset = f['simulation data']
                        r = np.array(dset["Initial Position"][:])
                        time_point = int(dset.attrs["Time index"])
                        phase = dset.attrs['Phase']


                ic_table[ic_counter] = {'path':os.path.join(root_folder, folder), 'ic':file[:-5], 'filament shape':r, 'phase':phase}


                ic_counter+=1
            
    return ic_table

In [118]:
folders = os.listdir(fil.homing_folder)
# folders = ['IC_31353']

ic_table = create_batchsim_table(folders, fil)


In [119]:
def run_homing_simulation_batch(fil = None, ic_table = None):
    """ Runs a batch/parallel simulation on a set of filament ICs specified using ic_table 
        for a given filament
        Inputs: filament object containing the filament corresponding to the current parameter value
                ic_table: List of ICs to simulate
                
        Returns: Nothing. Saves results of the IC simulations to disk.
    """
    #-----------------------------------------------------------------------------------------------------
    def run_ic_simulation(ic, path, filament_shape, phase):
#     """Run a filament simulation using specified parameters
#     """
        # Simulate each IC
        new_fil = activeFilament(dim = dim, Np = Np, radius = radius, b0 = b0, k = k, S0 = S0, D0 = D0, bc = bc)

        new_fil.simulate(Tf, Npts, save = True, overwrite = False, path = path, sim_type = 'point', 
        init_condition = {'filament':filament_shape}, 
        activity={'type':'square-wave','activity time scale':activity_timescale, 'duty_cycle':0.5, 'start phase':0},
                         create_subdirs=False)
    #-----------------------------------------------------------------------------------------------------
    # No:of activity cycles we want to simulate
    n_activity_cycles = 50

    bc = {0:'clamped', -1:'free'}

    dim = fil.dim
    Np = fil.Np
    radius = fil.radius
    b0 = fil.b0
    k = fil.k
    S0 = fil.S0
    D0 = 1.0
    activity_timescale = 750
    
    # Total simulation time
    Tf = activity_timescale*n_activity_cycles

    # Total simulation time
    # No:of time points saved
    time_step_save = 5
    Npts = int(Tf/time_step_save)
    
    # Run the Parallel/Batch simulations
    num_cores = multiprocessing.cpu_count()
    print(num_cores)
    num_cores = 12
    results = Parallel(n_jobs = num_cores, verbose=10)(delayed(run_ic_simulation)(ic_table[ic_index]['ic'], ic_table[ic_index]['path'], ic_table[ic_index]['filament shape'], ic_table[ic_index]['phase']) for ic_index in tqdm(ic_table.keys()))
    

In [120]:
run_homing_simulation_batch(fil = fil, ic_table=ic_table)

  0%|          | 0/100 [00:00<?, ?it/s]

16


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
 48%|████▊     | 48/100 [07:47<10:08, 11.69s/it][Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  8.1min
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed: 11.6min
100%|██████████| 100/100 [21:13<00:00, 12.74s/it]
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed: 21.7min
[Parallel(n_jobs=12)]: Done  88 out of 100 | elapsed: 25.7min remaining:  3.5min
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed: 27.9min finished
